# Übung 4.2 - Neuronale Netze am Beispiel Rattern
VO Maschinelles Lernen in der Produktion, WS2020/21, Moritz von Unold, Richard Lux, Felix Soest

#### In diesem Notebook soll das Verfahren Neuronale Netze anhand des Anwendungsbeispiels Rattern geübt werden.

Bei der Fertigung von Bauteilen treten manchmal störende Schwingungen auf, sog. Rattern. Dieses schädigt die Werkzeuge und führt zu einer niedrigeren Bauteilqualität.

Im Datensatz "Rattern" werden die Drehzahl der Spindel und die Tiefe des Schnitts gemessen. Es soll ein neuronales Netz erstellt werden welches vorhersagen kann bei welcher Drehzahl und Schnitt-Tiefe das sog. Rattern auftritt. 
### Data-Mining-Prozess:

![alt text](Prozess_Modellentwicklung_v2.png "Title")

### 0. Bibliohteken importieren

In [ ]:
# 0. Code-Block

# Importiere benötigte Bibliotheken
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.options.mode.chained_assignment = None
%matplotlib inline

### 1. Daten erfassen - Daten importieren

In [ ]:
# 1. Code-Block

# Erstelle eigene Zufallszahlen
my_seed = TODO

# Lade Datensatz und Ausgabe Zufallszahl
df_train = pd.read_csv("Trainingsdaten_Rattern.csv")
df_test = pd.read_csv("Testdaten_Rattern.csv")
df = df_train.append(df_test)
print("\nGewählte Zahl für Zufallszahlen: \t" + str(my_seed))

### 2. Daten erkunden

In [ ]:
# 2. Code-Block

# Ausgabe der letzten 5 Zeilen des Datensatzes
df.tail(5)

In [ ]:
# 3. Code-Block

# Ausgabe Klassenzugehörigkeit
fig = plt.figure(figsize=(20, 10))
surf = plt.scatter(
    df_train["Drehzahl Spindel"],
    df_train["Tiefe des Schnitts"],
    c=df_train["Rattern"],
    cmap=plt.cm.coolwarm,
    s=100,
    alpha=0.7,
    edgecolors="black",
)
plt.colorbar(surf)
plt.xlim(7750, 16250)
plt.ylim(0, 0.023)
plt.xlabel("Drehzahl Spindel")
plt.ylabel("Tiefe des Schnitts")
plt.title("Effekt des Ratterns über Schnitttiefe und Spindeldrehzahl")

### 3. Daten vorbereiten - Aufteilung in X und y, Normierung

In [ ]:
# 4. Code-Block

# Aufteilen in X und y
X_train_, y_train = (
    df_train[["Drehzahl Spindel", "Tiefe des Schnitts"]],
    df_train["Rattern"],
)
X_test_, y_test = (
    df_test[["Drehzahl Spindel", "Tiefe des Schnitts"]],
    df_test["Rattern"],
)

# Normiere Input-Parameter
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_train_), columns=X_train_.columns)
X_test = pd.DataFrame(scaler.transform(X_test_), columns=X_test_.columns)


# Ausgabe Maximum
print(
    "Vor Normierung:\t\tDrehzahl min/max: "
    + str("%.1f" % X_train_["Drehzahl Spindel"].min())
    + " / "
    + str("%.1f" % X_train_["Drehzahl Spindel"].max())
)
print(
    "Nach Normierung:\tDrehzahl min/max: "
    + str("%.1f" % X_train["Drehzahl Spindel"].min())
    + " / "
    + str("%.1f" % X_train["Drehzahl Spindel"].max())
)

### 4.1 Modelle bilden - Mögliche Hyperparameter anzeigen

In [ ]:
# 5. Code-Block

# Importieren des Modells
from sklearn.neural_network import MLPClassifier 

# Ausgabe möglicher Hyperparameter
print("\n Mögliche Hyperparameter (mit Standardeinstellungen) eines Neuronalen Netzes:")
MLPClassifier().get_params()

Beschreibung der Hyperparameter:
https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html

### 4.2 Modelle bilden - Optimale Hyperparameter mittels Gittersuche bestimmen

Um die optimalen Hyperparameter für das Neuronale Netz zu finden wird eine __Gittersuche__ durchgeführt:

TODO:
- Schreibe die zu variierenden Hyperparameter listenweise in ein Dictionary (6. Code-Block - Zeile 5-9)
- __Empfehlung:__ Verwende den solver lbfgs (nicht zwingend beste Möglichkeit): 'solver': ['lbfgs'], als random state deinen seed: random_state=my_seed
- Die Struktur eines Neuronalen Netzes (Anzahl an hidden layers sowie Neuronen je hidden layer) wird wie folgt angegeben:
    - hidden_layers_sizes = (3,) -> 1 hidden layer mit 3 Neuronen
    - hidden_layers_sizes = (3,2) -> 2 hidden layers mit 3 und 2 Neuronen
    - hidden_layers_sizes = (5,3,5) -> 3 hidden layers mit 5, 3 und 5 Neuronen
    - Zur Erinnerung: Die input layer hat soviele Neuronen wir Inputparameter

AUSGABE:
- Dauer und Anzahl an Kombinationen der Gittersuche sowie Modell mit besten Hyperparametern

In [ ]:
# 6. Code-Block

# Werte für die Gittersuche/Hyperparameter
from sklearn.model_selection import GridSearchCV
import time

start_timer = time.monotonic()
hyper_parameters = [{'hidden_layer_sizes': [TODO], 
                     TODO,
                     TODO,
                     TODO,
                     'random_state': [my_seed]}]

# Berechne Genauigkeit auf Validationsdaten für alle möglichen Kombinationen
mlp = MLPClassifier()
gridSearch = GridSearchCV(mlp, hyper_parameters, return_train_score=True, cv=5)
gridSearch = gridSearch.fit(X_train, y_train)
print(
    "\nDie Gittersuche ("
    + str(len(pd.DataFrame(gridSearch.cv_results_)))
    + " Kombinationen) hat "
    + str("%.1f" % (time.monotonic() - start_timer))
    + " Sekunden gedauert, bestes Modell:"
)
gridSearch.best_estimator_

In [ ]:
# 7. Code-Block

# Ausgabe der Ergebnis-Tabelle der Gittersuche - besten 15 Modelle
pd.DataFrame(gridSearch.cv_results_).sort_values(
    "mean_test_score", ascending=False
).head(15)

### 4.3 Modell bilden - Vorzugs-Modell erstellen und trainieren

Das Modell mit den optimalen Hyperparametern wird erstellt und mit den Trainingsdaten trainiert.

TODO.
- Schreibe die Code-Zeilen für die Genauigkeit des Modells auf den Trainingsdaten (7. Code-Block - Zeile 10) (Hilfe im Cheat-Sheet Modelle testen)

AUSGABE:
- Genauigkeit auf Trainingsdaten

In [ ]:
# 8. Code-Block

# Erstellen des Modells mit den besten Hyperparametern aus der gridSearch => gridSearch.best_estimator_
best_mlp = gridSearch.best_estimator_

# Ausgabe Genauigkeit auf Trainingsdaten
TODO

### 6.1 Modell testen & anwenden - Bewertung des Modells - Genauigkeit auf Testdaten

Um die Qualität/Güte des Modells zu bestimmen wird die Genauigkeit auf den Testdaten berechnet (Hilfe im Cheat-Sheet Modelle_testen).

In [ ]:
# 9. Code-Block

# Berechne Genauigkeit auf Testdaten
TODO

### 6.2 Modell testen & anwenden - Bewertung des Modells - Konfusionsmatrix

Um die Modellgüte visuell darzustellen wird eine Wahrheitsmatrix erstellt.

TODO:
- Schreibe die Code-Zeilen um die Wahrheitsmatrix des Modells auf den Testdaten anzuzeigen (Hilfe im Cheat-Sheet Modelle_testen).

AUSGABE:
- Konfusionsmatrix

In [ ]:
# 10. Code-Block

# Ausgabe Konfusionsmatrix
TODO

### 6.3 Modell testen & anwenden - Modell visualisieren

Da das Modell nur 2 Input-Parameter benötigt können wir alle möglichen Modell-Vorhersagen grafisch visualisieren.

In [ ]:
# 11. Code-Block

# Erstelle Grid für Modellausgabe
x = np.linspace(-0.1, 1.1, 200)
y = np.linspace(-0.1, 1.1, 200)

# Berechne Modellvorhersage
z = []
for x_ in x:
    for y_ in y:
        z.append(best_mlp.predict(np.asarray([y_, x_]).reshape(1, -1)))

# Ausgabe Modellvorhersage
X, Y = np.meshgrid(x, y)
Z = np.asarray(z).reshape(200, 200)
fig = plt.figure(figsize=(20, 12))
plt.title(
    "Modellvorhersage und Datensätze (Train = Kreis, Valid = Viereck, Test = Raute)"
)
plt.xlabel("Drehzahl Spindel")
plt.ylabel("Tiefe des Schnitts")
surf = plt.contourf(X, Y, Z, cmap=plt.cm.coolwarm)
plt.scatter(
    X_train["Drehzahl Spindel"],
    X_train["Tiefe des Schnitts"],
    c=y_train,
    marker="o",
    alpha=0.3,
    edgecolors="black",
    s=70,
)
plt.scatter(
    X_test["Drehzahl Spindel"],
    X_test["Tiefe des Schnitts"],
    c=y_test,
    marker="D",
    edgecolors="black",
    s=70,
)
plt.show()

### 6.4 Modell testen & anwenden - Modell analysieren

Ausgabe der trainierten Gewichte (Gewichtsmatrizen) des neuronalen Netzes.

In [ ]:
# 12. Code-Block

# Ausgabe der Gewichte
best_mlp.coefs_

### Verständnisfragen:

1. Vergleiche für dieses Problem (Rattern) die Modelle SVM und NN in Bezug auf: Aufwand der Erstellung/GridSearch, komplexität des Modells bzw. der Hyperparameter und das Ergebnis (Genauigkeit und Bild aus 6.3) -> Würdest du das nächste mal ein SVM-Modell oder ein NN-Modell wählen?
2. Wie viele Neuronen besitzt die output layer für dieses Problem? Wie kann daraus die resultierende Klasse gewählt werden?
3. Welche der folgenden Einstellungen haben einen Einfluss auf die Dauer der GridSearch?
    - Anzahl an variierenden Hyperparametern
    - Größe der Trainingsdaten
    - Anzahl an Splits der Cross-Validation (hier standardmässig auf 3 gesetzt)
    - Größe der Testdaten
    - Anzahl an Werten je Hyperparametern
    - Komplexität des gesuchten Zusammenhangs
    - Größe der Validierungsdaten